In [1]:
# Imports
import json
import requests
import time
import numpy as np
import pandas as pd

### Collecting Rank Data

Here we will be gathering rank data. I'll mostly just dump the data and deal with sorting it out more later on. That does mean the file will be large

In [2]:
class HaloData():
    def __init__(self,location=''):
        """
        This class manages data for the users as it gathers more of them
        
        """
        self.data = self.fromfile(location)
        self.gamemodes = {'4a2cedcc90984728886f60649896278d':"3v3",
                          '379f9ee592ec45d9b5e59f30236cab00':"2v2",
                          '548d864e8666430e91408dd2ad8fbfcd':'1v1'}
        # Key for website
        micro_key = ""
        self.header = {"Ocp-Apim-Subscription-Key": micro_key}
    
    def gamertags(self,sort='',nasort = ''):
        """
        Outputs all the usernames that have been collected so far.
        
        Parameters
            sort (String): Which column you want the data sorted by
        
        Return:
            (List): List of usernames
        
        """
        data = self.data
        if sort != '':
            data = self.data.sort_values(by=sort,ignore_index=True,na_position='first')
        elif nasort !='':
            data = self.data[self.data.isnull().any(axis=1)]
        
        return data['usernames']

    def getRanks(self,players):
        """
        Collects the MMR and rank for each player and stores it into the main table
        
        Parameters
            players (List of Strings): Up to 6 players that will get collected for
        
        Return:
            (List): All of the rank data for those players
        
        
        """
        # I don't know what to output here
        output = list()
        
        #Adding all the usernames into a single string
        player = ''
        for itm in players:
            player +='{},'.format(itm)
        player = player[:-1]

        # Loops through the gamemodes
        #The keys for the dictionary are the ID's that the API uses
        for itm in self.gamemodes.keys():
            # The request
            website = 'https://www.haloapi.com/stats/hw2/playlist/{0}/rating?players={1}'.format(itm,player)
            requestRaw = requests.get(website,headers= self.header)
            time.sleep(1)
            requestDict = json.JSONDecoder().decode(requestRaw.text)
            
            # Loops through each player and adds the data into the proper column
            for item in requestDict['Results']:
                name = item['Id']
                mask = np.where(self.gamertags() == name)[0]
                # Doesn't let you put dictionaries into a spot
                self.data.at[mask,self.gamemodes[itm]] = [item['Result']]
                
            output.append(requestDict)
        return output

    def createfile(self,name):
        data = self.data.sort_values(by='usernames',ignore_index=True)
        data.to_csv(name,index=False)
        return True
    def fromfile(self,name):
        return pd.read_csv(name)
    

In [7]:
halo_class = HaloData('ranks.csv')

In [8]:
# The list I'll be using
names = halo_class.gamertags(nasort='yes')
length = len(names)
# for loop with steps of 6
for i in range(0,length,6):
    try:
        # Basically checking that we aren't at the end
        if i < (int(length-6)):
            out = halo_class.getRanks(names[i:i+6])
            print(int(i/6),i,length-i)
        else:
            out = halo_class.getRanks(names[i:(i+(length%6)-1)])
    except KeyboardInterrupt:
        break
    # I know this is bad form but I just want to skip anyone who causes errors
    except:
        1+1

0 0 7


In [ ]:
halo_class.createfile('ranks.csv')

In [9]:
# This one dude
df = halo_class.data
df[df.isnull().any(axis=1)]

,usernames,1v1,2v2,3v3
90097,zzzzss,NaN,NaN,NaN
